In [4]:
from manapy.cuda.utils import patch_numpy_functions

patch_numpy_functions.patch()

from manapy.cuda.utils import (
  VarClass,
  GPU_Backend
)

import numpy as np
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import os

from timeit import default_timer as timer

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  return elapsed_time * 1000
  #print(f"{elapsed_time * 1000000:.5f} micros")

def init(dim, mesh_file_name, float_precision):
  GPU_Backend.float_precision = 'float64'
  if float_precision == 'single':
    GPU_Backend.float_precision = 'float32'
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)

  #running_conf = Struct(backend="numba", signature=True, cache=True, precision="double")
  #?create the mesh that will be used by domain
  MeshPartition(mesh_file_name, dim=dim, conf=running_conf, periodic=[0,0,0])
  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w)


class Solver():
  
  def __init__(self, var, vel, order, cft, diffusion = False, advection = False, Dxx = 0.0, Dyy = 0.0, Dzz = 0.0, dim = 2):

    self.var = var
    self.comm = self.var.comm
    self.domain = self.var.domain
    self.dim = self.var.dim
    self.float_precision = self.domain.float_precision
    
    self.u  = vel[0]
    self.v  = vel[1]
    self.w  = vel[2]

  

    self.advection = advection
    self.diffusion = diffusion
    self.order = np.int32(order)
    self.cfl   = np.float64(cft)
    self.Dxx   = np.float64(Dxx)
    self.Dyy   = np.float64(Dyy)
    self.Dzz   = np.float64(Dzz)

    self.backend = self.domain.backend
    self.signature = self.domain.signature


  
    #self.dt    = self.__stepper()
    


    self.var.__dict__["convective"] = np.zeros(self.domain.nbcells, dtype=self.float_precision)
    self.var.__dict__["dissipative"] = np.zeros(self.domain.nbcells, dtype=self.float_precision)
    self.var.__dict__["source"] = np.zeros(self.domain.nbcells, dtype=self.float_precision)
    

  


  #!only if advec or advecDiff
  def __explicit_convective(self):
      if self.order == 2:
          self.var.compute_cell_gradient()
      self._explicitscheme_convective(self.var.convective, self.var.cell, self.var.ghost, self.var.halo, self.u.face, self.v.face, self.w.face,
                                      self.var.gradcellx, self.var.gradcelly, self.var.gradcellz, self.var.gradhalocellx, 
                                      self.var.gradhalocelly, self.var.gradhalocellz, self.var.psi, self.var.psihalo, 
                                      self.domain.cells.center, self.domain.faces.center, self.domain.halos.centvol, 
                                      self.domain.faces.ghostcenter, self.domain.faces.cellid, self.domain.faces.normal, 
                                      self.domain.faces.halofid, self.domain.faces.name, 
                                      self.domain.innerfaces, self.domain.halofaces, self.domain.boundaryfaces, 
                                      self.domain.periodicboundaryfaces, self.domain.cells.shift, self.order)
  

  
  def stepper(self):
    d_t = self._time_step(self.u.cell, self.v.cell, self.w.cell, self.cfl, self.domain.faces.normal, self.domain.faces.mesure, 
                          self.domain.cells.volume, self.domain.cells.faceid, self.dim)
    self.dt = self.comm.allreduce(d_t, op=MPI.MIN)
    return  self.dt

  def compute_fluxes(self):
      #interpolate cell to node
      self.var.update_halo_value()
      self.var.update_ghost_value()
      self.__explicit_convective()
      

  def compute_new_val(self):
    self._update_new_value(self.var.cell, self.var.convective, self.var.dissipative, self.var.source, self.dt, self.domain.cells.volume)
  



dim=2
#mesh_file_name="/home/aben-ham/Desktop/work/stage/my_manapy/manapy/mesh/2D/carre.msh"
#mesh_file_name="/home/aben-ham/Desktop/work/stage/manapy/mesh/2D/square_larger.msh"
mesh_file_name="/home/aben-ham/Desktop/work/stage/my_manapy/gpu_accelerator/functions/square.msh"
diffusion=False
advection=True
order=2

domain, ne, u, v, w = init(dim=dim, mesh_file_name=mesh_file_name, float_precision='single')
S = Solver(ne, vel=(u, v, w), order=order, cft=0.8,
          diffusion=diffusion, advection=advection, Dxx=0.0, Dyy=0.0, Dzz=0.0, dim = dim)



Patch the numpy functions np.array, np.zeros, np.ones, np.empty, np.full
Reading gmsh file ...
Create empty
Create empty
Create full
Create ones
Create ones
Create ones
Create ones
Create ones
Create zeros
Create zeros
Create zeros
Saving partition files ...
Create asarray
Create asarray
Number of Cells: 258030
Number of Vertices: 129684
Local domain contruction ...
Create array
Create zeros
Create zeros
Create ones
Create ones
Create empty
Create empty
Create ones
Create ones
Create zeros
Create zeros
Create zeros
Create zeros
Create zeros
Create zeros
Create zeros
Create asarray
Create asarray
Create ones
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create array
Create a

In [5]:
type(domain._cells._cellnid)

numpy.ndarray

In [7]:
from manapy.cuda.manapy.ast.cuda_ast_utils import (
  get_kernel_facetocell,
  get_kernel_celltoface,

)

from manapy.cuda.manapy.comms.cuda_communication import (
  get_kernel_define_halosend
)

from manapy.cuda.manapy.ast.cuda_functions2d import (
  get_kernel_centertovertex_2d,
  get_kernel_face_gradient_2d,
  get_kernel_cell_gradient_2d,
  get_kernel_barthlimiter_2d,
)

from manapy.cuda.manapy.solvers.advec.cuda_fvm_utils import (
  get_kernel_explicitscheme_convective_2d,
  get_kernel_time_step,
  get_kernel_update_new_value,
)

from manapy.cuda.manapy.util_kernels import get_kernel_assign

funcs = [
  [get_kernel_facetocell(), None],
  [get_kernel_celltoface(), None],
  [get_kernel_define_halosend(), None],
  [get_kernel_centertovertex_2d(), None],
  [get_kernel_face_gradient_2d(), None],
  [get_kernel_cell_gradient_2d(), None],
  [get_kernel_barthlimiter_2d(), None],
  [get_kernel_explicitscheme_convective_2d(), None],
  [get_kernel_time_step(), None],
  [get_kernel_update_new_value(), None],

]

i = 0
Variable._facetocell = funcs[i][0]; i += 1
Variable._celltoface = funcs[i][0]; i += 1
Variable._define_halosend = funcs[i][0]; i += 1
#2d
Variable._func_interp   = funcs[i][0]; i += 1
Variable._face_gradient = funcs[i][0]; i += 1
Variable._cell_gradient = funcs[i][0]; i += 1
Variable._barthlimiter  = funcs[i][0]; i += 1

S._explicitscheme_convective = funcs[i][0]; i += 1
S._time_step = funcs[i][0]; i += 1
S._update_new_value = funcs[i][0]; i += 1
S._time_step = get_kernel_time_step()

compile kernel_facetocell to cuda => signature=void(float32[:], float32[:], int32[:,:], int32)
compile kernel_celltoface to cuda => signature=void(float32[:], float32[:], float32[:], float32[:], int32[:,:], int32[:], int32[:], int32[:], int32[:])
compile kernel_define_halosend to cuda => signature=void(float32[:], float32[:], int32[:])
compile kernel_centertovertex_2d to cuda => signature=void(float32[:], float32[:], float32[:], float32[:], float32[:,:], float32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], int32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], int32[:], float32[:,:], float32[:])
compile kernel_face_gradient_2d to cuda => signature=void(float32[:], float32[:], float32[:], float32[:], int32[:,:], int32[:,:], float32[:,:], int32[:], float32[:,:], float32[:,:], float32[:,:], float32[:], float32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:,:], float32[:], float32[:], f

In [ ]:
def test_1():
  print('==================================')
  print('==================================')
  u.interpolate_facetocell()
  print('==================================')
  print('==================================')
  S.compute_fluxes()
  print('==================================')
  print('==================================')
  S.compute_new_val()

In [9]:
a = -1 * VarClass(np.ones(5, dtype=np.int32))
a

VarClass<int32, (5,)>


VarClass([-1, -1, -1, -1, -1], dtype=int32)

In [6]:
a = VarClass(ne.domain.cells.faceid)
b = VarClass(a)
a.type_shape
b.type_shape

VarClass<int32, (258030, 4)>


'VarClass<int32, (258030, 4)>'

In [26]:
VarClass(ne.domain.cells.faceid)

AttributeError: 'VarClass' object has no attribute 'type_shape'

In [20]:
ne.domain.cells.faceid.shape
ne.domain.faces.normal.shape, ne.domain.faces.mesure.shape
ne.domain.cells.volume.type_shape
ne.domain.cells.faceid.type_shape

AttributeError: 'VarClass' object has no attribute 'type_shape'

In [8]:
S.stepper()

CUDADispatcher(<function get_kernel_time_step.<locals>.kernel_time_step at 0x7b8a6418a820>) is called
1 => VarClass<float32, (258030,)>
2 => VarClass<float32, (258030,)>
3 => VarClass<float32, (258030,)>
<class 'numpy.float64'>
5 => VarClass<float32, (387713, 3)>
6 => VarClass<float32, (387713,)>
7 => VarClass<float32, (258030,)>


AttributeError: 'VarClass' object has no attribute 'type_shape'